# Folder Structure
DI_centre_structured\

.\DI_CAMERA_P3225 -> Final -> [Patient Name] -> [2/3] Meters -> With Blankets | Without Blankets -> Hold Breath | Relaxed -> [Name][distance number].mp4

.\DI_THERMAL_FUR -> [patient number] -> [distance] -> H|R -> B | WOB -> [patient number]_[H/R]_[distance]m.mp4

In [17]:
import os
import cv2
import json

In [18]:
p3225_file_path = {
    "first level" : ["Final"],
    "second level" : ["Arun"],
    "third level" : [2, 3],
    "fourth level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"}, 
    "fifth level" : {"H" : "Hold Breath",
                    "R": "Relaxed"} 
}

fur_file_path = {
    "first level" : None,
    "second level" : [1],
    "third level" : [2, 3],
    "fourth level" : { "H": "Hold Breath",
                        "R": "Relaxed"}, 
    "fifth level" : { "B": "With Blankets",
                        "WOB": "Without Blankets"} 
}

Solution:
1) concatenate all possible filepaths
2) get all mp4 files and filter out unwanted files using regex

Workflows:
<br>
- A function to check for folder structure
<br>
- Manually check each folder for missing videos, keep record of results
<br>
- One function to find all mp4 files
<br>
- One function to check to convert all mp4 files into frames (linspace to pick out frames)
<br>
- zip frames folder to save space

Notes:
- checked the file you sent and it looks like you did something similar

Studying Video Properties:
- using media play, video length = 7s and frame rate = 18.563

In [19]:
"""
    A function to convert mp4 files to frames
    video length = 7s
    frame rate = 18.563
"""

def convert_mp4_to_frames(mp4_file:str, output_dir:str):
    video = cv2.VideoCapture(mp4_file)
    os.makedirs(output_dir, exist_ok=True)
    frame_count = 0
    not_end_of_vid = True

    while not_end_of_vid:
        ret, frame = video.read()
        
        if not ret:
            not_end_of_vid = False
            break

        frame_file = os.path.join(output_dir, f"frame{frame_count:05d}.jpg")
        cv2.imwrite(frame_file, frame)
        frame_count += 1

    video.release()

    print(f"Conversion complete: {frame_count} frames saved in {output_dir}")

In [22]:
"""
    display video properties using opencv
"""

def get_video_properties(mp4_file: str) -> dict:
    video_properties = {}
    video = cv2.VideoCapture(mp4_file)
    old_fps = int(video.get(cv2.CAP_PROP_FPS))
    number_of_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    video_properties["old fps"] = old_fps
    video_properties["frames"] = number_of_frames


    return video_properties

In [23]:
""" Sample Usage of functions (main function) """
mp4_file = "./DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Breathed/Arun2.mp4"
output_directory = "./DI_CAMERA_P3225/Final/Arun/2 Meters/With Blankets/Breathed/frames"

# convert_mp4_to_frames(input_file, output_directory)
video_properties = get_video_properties(mp4_file)
video_properties_json = json.dumps(video_properties, indent=2)
print(video_properties_json)

{
  "old fps": 18,
  "frames": 143
}
